# Pré-análise da amostra

Analise de uma amostra do dataset **GDELT 2.0: Our Global World in Realtime**:
- Quantidade de linhas
- Quantidade de linhas duplicadas
- Analise das colunas

In [1]:
import funcoes_s3

In [2]:
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import warnings
warnings.filterwarnings("ignore")

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [3]:
from pyspark.sql import SparkSession

# build our own SparkSession
myspark = SparkSession\
    .builder\
    .appName("BigData")\
    .config("spark.sql.shuffle.partitions",6)\
    .config("spark.sql.repl.eagereval.enabled",True)\
    .getOrCreate()

22/04/30 16:23:29 WARN Utils: Your hostname, SBS-RENPOR resolves to a loopback address: 127.0.1.1; using 172.30.197.0 instead (on interface eth0)
22/04/30 16:23:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/30 16:23:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/30 16:23:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# print SparkSession object
print(myspark)

In [5]:
# check it, including the link
myspark

## Download dos ficheiros de amostra

In [17]:
data_path = "/mnt/c/Users/Rene Porto/OneDrive - ISCTE-IUL/Projecto Big Data/Data/"

In [7]:
filenames = pd.read_csv("/mnt/c/Users/Rene Porto/OneDrive - ISCTE-IUL/Projecto Big Data/filename.csv")

day_20190412 = filenames[filenames['year_month_day']==20190412]
day_20190412.sample(5)

,filename,size_b,year,year_month,year_month_day
143031,20190412103000.gkg.csv,25805635,2019,201904,20190412
143083,20190412233000.gkg.csv,30725134,2019,201904,20190412
143056,20190412164500.gkg.csv,38179531,2019,201904,20190412
143071,20190412203000.gkg.csv,34977913,2019,201904,20190412
143040,20190412124500.gkg.csv,30679387,2019,201904,20190412


In [8]:
day_20190412 = day_20190412['filename']
print(f"Quantidade de linhas para o dia 12/04/2019: {day_20190412.count()}")

In [10]:
# Configuração do S3
s3_resource = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
bucket_name = 'gdelt-open-data' 
prefix = "v2/gkg/"

In [11]:
funcoes_s3.download_list_objects(s3_resource, bucket_name, prefix, day_20190412, data_path)

Already downloaded: 20190412000000.gkg.csv
0
Already downloaded: 20190412001500.gkg.csv
0
Already downloaded: 20190412003000.gkg.csv
0
Already downloaded: 20190412004500.gkg.csv
0
Already downloaded: 20190412010000.gkg.csv
0
Already downloaded: 20190412011500.gkg.csv
0
Already downloaded: 20190412013000.gkg.csv
0
Already downloaded: 20190412014500.gkg.csv
0
Already downloaded: 20190412020000.gkg.csv
0
Already downloaded: 20190412021500.gkg.csv
0
Already downloaded: 20190412023000.gkg.csv
0
Already downloaded: 20190412024500.gkg.csv
0
Already downloaded: 20190412030000.gkg.csv
0
Already downloaded: 20190412031500.gkg.csv
0
Already downloaded: 20190412033000.gkg.csv
0
Already downloaded: 20190412034500.gkg.csv
0
Already downloaded: 20190412040000.gkg.csv
0
Already downloaded: 20190412041500.gkg.csv
0
Already downloaded: 20190412043000.gkg.csv
0
Already downloaded: 20190412044500.gkg.csv
0
Already downloaded: 20190412050000.gkg.csv
0
Already downloaded: 20190412051500.gkg.csv
0
Already do

In [34]:
schema = StructType() \
      .add("GKGRECORDID", StringType(), True) \
      .add("DATE", IntegerType(), True) \
      .add("SourceCollectionIdentifier", IntegerType(), True) \
      .add("SourceCommonName", StringType(), True) \
      .add("DocumentIdentifier", StringType(), True) \
      .add("Counts", StringType(), True) \
      .add("V2Counts", StringType(), True) \
      .add("Themes", StringType(), True) \
      .add("V2Themes", StringType(), True) \
      .add("Locations", StringType(), True) \
      .add("V2Locations", StringType(), True) \
      .add("Persons", StringType(), True) \
      .add("V2Persons", StringType(), True) \
      .add("Organizations", StringType(), True) \
      .add("V2Organizations", StringType(), True) \
      .add("V2Tone", StringType(), True) \
      .add("Dates", StringType(), True) \
      .add("GCAM", StringType(), True) \
      .add("SharingImage", StringType(), True) \
      .add("RelatedImages", StringType(), True) \
      .add("SocialImageEmbeds", StringType(), True) \
      .add("SocialVideoEmbeds", StringType(), True) \
      .add("Quotations", StringType(), True) \
      .add("AllNames", StringType(), True) \
      .add("Amounts", StringType(), True) \
      .add("TranslationInfo", StringType(), True) \
      .add("Extras", StringType(), True)

In [36]:
amostra_20190412 = myspark.read.format("csv") \
      .options(header=False, delimiter='\t') \
      .schema(schema) \
      .load(data_path)

In [37]:
amostra_20190412.printSchema()

root
 |-- GKGRECORDID: string (nullable = true)
 |-- DATE: integer (nullable = true)
 |-- SourceCollectionIdentifier: integer (nullable = true)
 |-- SourceCommonName: string (nullable = true)
 |-- DocumentIdentifier: string (nullable = true)
 |-- Counts: string (nullable = true)
 |-- V2Counts: string (nullable = true)
 |-- Themes: string (nullable = true)
 |-- V2Themes: string (nullable = true)
 |-- Locations: string (nullable = true)
 |-- V2Locations: string (nullable = true)
 |-- Persons: string (nullable = true)
 |-- V2Persons: string (nullable = true)
 |-- Organizations: string (nullable = true)
 |-- V2Organizations: string (nullable = true)
 |-- V2Tone: string (nullable = true)
 |-- Dates: string (nullable = true)
 |-- GCAM: string (nullable = true)
 |-- SharingImage: string (nullable = true)
 |-- RelatedImages: string (nullable = true)
 |-- SocialImageEmbeds: string (nullable = true)
 |-- SocialVideoEmbeds: string (nullable = true)
 |-- Quotations: string (nullable = true)
 |-- A

In [27]:
# Para tornar a visualização do Dataframe melhor
myspark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [38]:
amostra_20190412

GKGRECORDID,DATE,SourceCollectionIdentifier,SourceCommonName,DocumentIdentifier,Counts,V2Counts,Themes,V2Themes,Locations,V2Locations,Persons,V2Persons,Organizations,V2Organizations,V2Tone,Dates,GCAM,SharingImage,RelatedImages,SocialImageEmbeds,SocialVideoEmbeds,Quotations,AllNames,Amounts,TranslationInfo,Extras
20190412183000-0,null,1,tpr.org,https://www.tpr.o...,null,null,TAX_WORLDMAMMALS;...,SOC_POINTSOFINTER...,1#Germany#GM#GM#5...,1#Germany#GM#GM##...,fritz breithaupt;...,"Fritz Breithaupt,...",experimental huma...,Experimental Huma...,"2.62828535669587,...",1#0#0#1909#1141,"wc:1477,c1.3:1,c1...",null,null,null,null,null,"Dark Sides,543;Ex...","2,sides,5405;",null,<PAGE_LINKS>https...
20190412183000-1,null,1,oudaily.com,http://www.oudail...,null,null,LEADER;TAX_FNCACT...,"TRIAL,1153;USPEC_...","2#California, Uni...","2#Oklahoma, Unite...",mitch mcconnell;k...,"Mitch Mcconnell,1...",senate majority l...,Senate Majority L...,"1.5748031496063,4...",null,"wc:240,c1.2:2,c12...",https://bloximage...,null,null,null,null,"Kendra Horn,162;I...","20,states launchi...",null,<PAGE_LINKS>https...
20190412183000-2,null,1,northernpublicrad...,https://www.north...,null,null,TAX_FNCACT;TAX_FN...,TAX_ETHNICITY_AME...,1#United States#U...,1#Americans#US#US...,amanda kolson hur...,Amanda Kolson Hur...,null,null,-3.03030303030303...,null,"wc:85,c1.3:1,c12....",null,null,null,null,null,Amanda Kolson Hur...,null,null,<PAGE_AUTHORS>Ete...
20190412183000-3,null,1,english.wafa.ps,http://english.wa...,null,null,TAX_ETHNICITY;TAX...,TAX_WORLDLANGUAGE...,"4#Ramallah, West ...",1#West Bank#WE#WE...,null,null,null,null,-9.49720670391061...,null,"wc:158,c1.1:2,c12...",null,null,null,https://youtube.c...,null,Several Palestini...,null,null,null
20190412183000-4,null,1,romseyadvertiser....,https://www.romse...,PROTEST#200#stude...,PROTEST#200#stude...,TAX_FNCACT;TAX_FN...,EPU_POLICY_GOVERN...,"4#Bedford, Boroug...",1#United Kingdom#...,theresa may;cyrus...,"Theresa May,655;C...",parliament square...,Parliament Square...,-5.10204081632653...,null,"wc:647,c1.3:34,c1...",https://www.romse...,https://image.ass...,null,null,732|60||Theresa M...,"Youth Strike,200;...","4,Climate movemen...",null,<PAGE_PRECISEPUBT...
20190412183000-5,null,1,dailymail.co.uk,https://www.daily...,KILL#45##5#Khuzes...,KILL#45##5#Khuzes...,TAX_ETHNICITY;TAX...,"KILL,1358;WATER_S...","5#Lorestan, Lores...","4#Ahvaz, Khuzesta...",javad zarif;ghola...,"Javad Zarif,2334;...",national disaster...,National Disaster...,-7.17391304347826...,4#3#19#0#1258;4#3...,"wc:423,c1.2:2,c12...",https://i.dailyma...,null,null,null,null,Ali Asghar Peivan...,"70,centimetres,67...",null,<PAGE_AUTHORS>Afp...
20190412183000-6,null,1,iheart.com,https://theriver1...,null,null,TAX_FNCACT;TAX_FN...,"MEDIA_SOCIAL,338;...","2#New York, Unite...","3#Greene County, ...",null,null,cairo police depa...,"Facebook,338",-2.66666666666667...,null,"wc:139,c12.1:8,c1...",https://i.iheart....,null,null,null,null,"New York,127;New ...",null,null,<PAGE_LINKS>https...
20190412183000-7,null,1,iheart.com,https://alt96va.i...,null,null,TAX_WORLDFISH;TAX...,TAX_WORLDFISH_ROC...,"2#Virginia, Unite...","2#Virginia, Unite...",null,null,washington post,null,-4.93827160493827...,null,"wc:78,c12.1:3,c12...",https://i.iheart....,null,null,https://youtube.c...,null,"Washington Post,190",null,null,<PAGE_LINKS>https...
20190412183000-8,null,1,northernpublicrad...,https://www.north...,null,null,TAX_FNCACT;TAX_FN...,"WB_1406_DISEASES,...",1#Japan#JA#JA#36#...,1#Guatemalan#GT#G...,anurima ghosh das...,Anurima Ghosh Das...,null,null,"1.76916596461668,...",4#3#29#0#2156,"wc:1090,c1.1:1,c1...",null,http://media.npr....,null,null,4049|31||Recipes ...,Chef Anumitra Gho...,"2,food writers wh...",null,<PAGE_LINKS>http:...
20190412183000-9,null,1,iheart.com,https://theriver1...,KILL#25##3#Atlant...,KILL#25##3#Atlant...,SOC_GENERALCRIME;...,"KILL,1207;TAX_WOR...","2#Georgia, United...","2#Texas, United S...",christopher goffa...,Christopher Goffa...,los

22/04/30 22:46:39 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 13241835 ms exceeds timeout 120000 ms
22/04/30 22:46:40 WARN SparkContext: Killing executors is not supported by current scheduler.
